# Model Deployment with Flask

In this notebook, we’ll learn how to deploy a trained machine learning model as a web API using **Flask**, a lightweight Python web framework.

### Objectives
- Understand the basics of Flask for serving models.
- Create a simple Flask API endpoint.
- Load a trained model and return predictions via API.
- Test the Flask app locally using `requests` or `curl`.

In [1]:
# Install dependencies (if running locally)
# !pip install flask scikit-learn joblib

## Step 1: Train and Save a Sample Model

We’ll train a simple Logistic Regression model on the Iris dataset and save it for deployment.

In [2]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import joblib

# Load dataset
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# Train model
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# Save model
joblib.dump(model, 'iris_model.pkl')

## Step 2: Create Flask App for Deployment

We will build a simple Flask app that loads the model and provides a `/predict` API endpoint.

In [3]:
from flask import Flask, request, jsonify
import numpy as np

app = Flask(__name__)

# Load model
model = joblib.load('iris_model.pkl')

@app.route('/')
def home():
    return "<h2>ML Model Deployment with Flask</h2><p>Use the /predict endpoint to get predictions.</p>"

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    features = np.array(data['features']).reshape(1, -1)
    prediction = model.predict(features)
    class_name = iris.target_names[prediction[0]]
    return jsonify({'prediction': int(prediction[0]), 'class': class_name})

if __name__ == '__main__':
    app.run(debug=True)

## Step 3: Testing the API

You can run this Flask app and test it by sending a POST request.

### Example request using `requests` library:

In [4]:
# Example request (run this in another cell after starting Flask server)
# import requests
# sample_data = {"features": [5.1, 3.5, 1.4, 0.2]}
# response = requests.post('http://127.0.0.1:5000/predict', json=sample_data)
# print(response.json())

## Step 4: Notes on Deployment

- You can use **Gunicorn** or **uWSGI** with **Nginx** for production deployment.
- Flask can be containerized with **Docker**.
- For cloud deployment, you can use **AWS Elastic Beanstalk**, **Render**, or **Heroku**.

Example Dockerfile:
```dockerfile
FROM python:3.9
WORKDIR /app
COPY . .
RUN pip install flask scikit-learn joblib
CMD ["python", "app.py"]
```